In [0]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image
import random
from random import randint
from keras_preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.style.use ('dark_background')

In [0]:
# Funkcija za dobijanje 64x64 isecka originalne slike
# Koristim je za originalne test slike koje treba izdeliti

def izdeli_sliku (put_slike, put_izlaza):
    
    slika = Image.open(put_slike)
    ime_slike = str.split (put_slike, sep = os.path.sep)[-1]
    sir_slike, vis_slike = slika.size
    
    print (ime_slike)
    
    for i in range (0, sir_slike, 64):
        
        for j in range (0, vis_slike, 64):
            
            if i + 64 < sir_slike:
                k = i + 64
            else:
                i = sir_slike - 64 - 1
                k = sir_slike - 1
            
            if j + 64 < vis_slike:
                l = j + 64
            else:
                j = vis_slike - 64 - 1
                l = vis_slike - 1
            
            
            kontura = (i, j, k, l)
            isecak = slika.crop (kontura)
            
            
            try:
              isecak.save (put_izlaza + os.path.sep + str.split (ime_slike, sep = '.')[-2] + "_%s_" % str(i) + "%s.jpg" % str(j))
                
            except:
                print ('Nije uspelo cuvanje slike!')
            


In [0]:
# Proba funkcije
'''
koord = [540, 726]
put_slike = os.path.join ('..', 'data', 'Test', 'test1.jpg')
put_izlaza = os.path.join ('..', 'data', 'Test', 'test1_izdeljena')

izdeli_sliku (put_slike, put_izlaza)
'''

"\nkoord = [540, 726]\nput_slike = os.path.join ('..', 'data', 'Test', 'test1.jpg')\nput_izlaza = os.path.join ('..', 'data', 'Test', 'test1_izdeljena')\n\nizdeli_sliku (put_slike, put_izlaza)\n"

In [0]:
# Koriscenje ugradjene funkcije za generisanje novih podataka


osobine = ImageDataGenerator (
        rotation_range = 180,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        fill_mode = 'nearest')

# Generisemo samo slike walda, jer su klase neizbalansirane
putanja_slika = os.path.join ('..', 'data', 'Slike', 'images', '64', 'waldo')
gcolab_putanja_slika = os.path.join ('Hey-Waldo', '64')
gcolab_izlaz = os.path.join ('Hey-Waldo', '64', 'gen_waldo')

# Cuvam u gen_ direktorijumu
generator = osobine.flow_from_directory(
    directory = gcolab_putanja_slika,
    save_to_dir = gcolab_izlaz,
    save_prefix = 'gen',
    save_format = 'jpeg',
    target_size = (64, 64),
    color_mode = "rgb",
    batch_size = 4,
    class_mode = 'categorical',
    classes = ['waldo'],
    shuffle = True,
    seed=3
)

granica = 1300

# Pokrenuti ako je potrebno generisati nove slike

for i in range (granica):
    generator.next()


Found 39 images belonging to 1 classes.


In [0]:
# Kreiranje numpy nizova od dobijenih slika

valdo_putanja = os.path.join ('..', 'data', 'Slike', 'images', '64', 'waldo', '')
ne_valdo_putanja = os.path.join ('..', 'data', 'Slike', 'images', '64', 'notwaldo', '')

gcolab_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'gen_waldo', '')
gcolab_ne_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'notwaldo', '')


valdo = np.array([np.array(plt.imread (gcolab_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_valdo_putanja)])
ne_valdo = np.array([np.array(plt.imread (gcolab_ne_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_ne_valdo_putanja)])


In [0]:
# Kreiranje pandas DataFrame-a od dobijenih nizova

csv_putanja = os.path.join ('..', 'data', 'CSV', '')

gcolab_csv_putanja = os.path.join ('CSV', '')

podaci = []

for slika in valdo:
    podaci.append (slika.flatten ('F'))

df1 = pd.DataFrame (podaci)
df1 ['Waldo'] = 1

# Odkomentarisati po potrebi
#df1.to_csv (csv_putanja + 'waldo64.csv', index = False)


In [0]:
# Isto ovo za slike na kojima nije waldo

podaci = []

for slika in ne_valdo:
    podaci.append (slika.flatten ('F'))

df2 = pd.DataFrame (podaci)
df2 ['Waldo'] = 0

# Odkomentarisati po potrebi
#df2.to_csv (csv_putanja + 'notwaldo64.csv', index = False)


In [0]:
# Objedinjavanje oba DataFrame-a i kreiranje objedninjenog .csv fajla
# Pre objedinjavanja mesamo vrste objedinjenog DataFrame-a

df = [df1, df2]
df = pd.concat (df)
df = df.apply(np.random.permutation, axis=0)

df.to_csv (gcolab_csv_putanja + 'allwaldo64.csv', index = False)
